# Notebook 2 - SQL avec vraies bases de données
## Analyse e-commerce avec PostgreSQL en ligne




### 🎯 Objectifs pédagogiques
- Connecter Python à une vraie base de données PostgreSQL
- Écrire des requêtes SQL complexes sur des données réelles
- Implémenter des analyses RFM avec SQL
- Intégrer SQL et pandas pour des analyses avancées
- Gérer les connexions et la sécurité

### 🛍️ Contexte du projet
Vous analysez les données d'un vrai dataset e-commerce (Brazilian E-Commerce Public Dataset) hébergé sur une base PostgreSQL.

Objectif : créer une segmentation clientèle pour optimiser les campagnes marketing.


## Partie 1 : Connexion à la base de données réelle

### 🔧 Installation et configuration


# Installation des dépendances


```
pip install psycopg2-binary sqlalchemy pandas python-dotenv
```




In [ ]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
from sqlalchemy import text
from dotenv import load_dotenv

### 🌐 Base de données PostgreSQL gratuite (ElephantSQL)

**Option 1 : ElephantSQL (20MB gratuit)**
1. Créez un compte sur [elephantsql.com](https://www.elephantsql.com/)
2. Créez une instance "Tiny Turtle" (gratuite)
3. Récupérez vos credentials

**Option 2 : Supabase (500MB gratuit)**
1. Créez un compte sur [supabase.com](https://supabase.com/)
2. Créez un nouveau projet
3. Récupérez l'URL de connexion PostgreSQL

In [116]:
# Configuration de connexion (à adapter selon votre provider)
DATABASE_CONFIG = {
    'host': 'aws-0-eu-west-3.pooler.supabase.com',  #  host Supabase
    'database': 'postgres',
    'user': 'postgres.najrxlrneiiweukjgyea',
    'password': '0000aaa111ZZZ!?',
    'port': 5432,
}


# Création de l'engine SQLAlchemy
engine = create_engine(
    f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@"
    f"{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
)

# Test de connexion
def test_connection():
    """
    Testez votre connexion à la base

    Étapes :
    1. Utilisez pd.read_sql() pour exécuter "SELECT version()"
    2. Affichez la version PostgreSQL
    3. Gérez les erreurs de connexion
    """
    try:
        df = pd.read_sql('SELECT version();', engine)
        print('Connexion Ok | Version : ', df.iloc[0,0])
    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return False
    return True
test_connection()

Connexion Ok | Version :  PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-bit


True


## Partie 2 : Import du dataset e-commerce

### 📊 Dataset Brazilian E-Commerce
Nous utilisons le célèbre dataset Olist (100k commandes réelles).

**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [ ]:
### 📥 Import des données via API

import requests
import zipfile
import io

# def download_olist_dataset():
#     """
#     Télécharge le dataset Olist depuis Kaggle API

#     Alternative : Utilisez l'API publique de l'IBGE (Institut brésilien)
#     pour des données e-commerce synthétiques mais réalistes
#     """

#     # URL des données publiques brésiliennes
#     IBGE_API = "https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce"

#     # Récupération des données de villes (pour la géolocalisation)
#     cities_url = f"{IBGE_API}/localidades/municipios"

#     try:
#         response = requests.get(cities_url)
#         cities_data = response.json()

#         # Convertir en DataFrame
#         cities_df = pd.DataFrame(cities_data)

#         # Votre code pour nettoyer et structurer
#         # Créez des données e-commerce réalistes basées sur ces villes

#         return cities_df
#     except Exception as e:
#         print(f"Erreur API IBGE : {e}")
#         return None
#     pass

# Génération de données e-commerce réalistes
# def generate_ecommerce_data(cities_df, n_customers=10000):
#     """
#     Génère des données e-commerce réalistes

#     Étapes guidées :
#     1. Sélectionnez 50 villes brésiliennes aléatoirement
#     2. Créez des clients avec distribution réaliste
#     3. Générez des commandes avec saisonnalité
#     4. Ajoutez des produits avec catégories cohérentes
#     5. Calculez des prix et frais de port basés sur la distance
#     """
#     pass







olist_order_payments_dataset.csv chargé : 103886 lignes, 5 colonnes
olist_order_reviews_dataset.csv chargé : 99224 lignes, 7 colonnes
olist_products_dataset.csv chargé : 32951 lignes, 9 colonnes
olist_customers_dataset.csv chargé : 99441 lignes, 5 colonnes
olist_sellers_dataset.csv chargé : 3095 lignes, 4 colonnes
olist_orders_dataset.csv chargé : 99441 lignes, 8 colonnes
olist_geolocation_dataset.csv chargé : 1000163 lignes, 5 colonnes
product_category_name_translation.csv chargé : 71 lignes, 2 colonnes
olist_order_items_dataset.csv chargé : 112650 lignes, 7 colonnes


In [31]:
# Mise en DataFrame des csv
# Liste pour stockers les dataframes
df_list = {}
# Dossier avec les csv
data_folder = "data"
# Boucle sur les fichiers du dossier 
for csv in os.listdir(data_folder):
    if csv.endswith(".csv"): # Filtre les fichiers au format csv
        file_path = os.path.join(data_folder, csv)
        df = pd.read_csv(file_path) # Lecture des csv
        # Stockage dans le dictionnaire df_list
        name = os.path.splitext(csv)[0]
        df_list[name] = df
        print(f'{csv} chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes')

olist_order_payments_dataset.csv chargé : 103886 lignes, 5 colonnes
olist_order_reviews_dataset.csv chargé : 99224 lignes, 7 colonnes
olist_products_dataset.csv chargé : 32951 lignes, 9 colonnes
olist_customers_dataset.csv chargé : 99441 lignes, 5 colonnes
olist_sellers_dataset.csv chargé : 3095 lignes, 4 colonnes
olist_orders_dataset.csv chargé : 99441 lignes, 8 colonnes
olist_geolocation_dataset.csv chargé : 1000163 lignes, 5 colonnes
product_category_name_translation.csv chargé : 71 lignes, 2 colonnes
olist_order_items_dataset.csv chargé : 112650 lignes, 7 colonnes


**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [44]:
### 🗃️ Création des tables SQL
def create_tables():
    """
    Créez les tables dans PostgreSQL

    Tips :
    - Utilisez des SERIAL pour les IDs auto-increment
    - Ajoutez des index sur les clés étrangères
    - Incluez des contraintes de validation
    """

    create_customers = """
    CREATE TABLE IF NOT EXISTS customers (
        customer_id SERIAL PRIMARY KEY,
        customer_city VARCHAR(100),
        customer_state VARCHAR(2)
    );
    """
    create_orders = """
    CREATE TABLE IF NOT EXISTS orders (
        order_id SERIAL PRIMARY KEY,
        customer_id INT NOT NULL,
        order_status VARCHAR(20) CHECK (order_status IN (
            'created', 'shipped', 'delivered', 'canceled'
        )),
        order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        order_delivered_date TIMESTAMP,
        CONSTRAINT fk_customer
            FOREIGN KEY(customer_id)
                REFERENCES customers(customer_id)
        );
    """
    create_order_items = """ 
    CREATE TABLE IF NOT EXISTS order_items (
        order_id SERIAL PRIMARY KEY,
        product_id INT NOT NULL,
        seller_id INT,
        price NUMERIC(10,2) CHECK (price >= 0),
        freight_value NUMERIC(10,2) CHECK (freight_value >= 0),
        CONSTRAINT fk_product
            FOREIGN KEY(product_id)
                REFERENCES products(product_id),
        CONSTRAINT fk_seller
            FOREIGN KEY(seller_id)
                REFERENCES sellers(seller_id),
        CONSTRAINT fk_order
            FOREIGN KEY(order_id)
                REFERENCES orders(order_id)
    );
    """
    create_products = """ 
    CREATE TABLE IF NOT EXISTS products (
        product_id SERIAL PRIMARY KEY,
        product_category VARCHAR(50),
        product_weight_g NUMERIC CHECK (product_weight_g >=0)  
    );
    """
    create_sellers = """
    CREATE TABLE IF NOT EXISTS sellers (
        seller_id SERIAL PRIMARY KEY,
        seller_city VARCHAR(50),
        seller_state VARCHAR(2)
    );
    """

    # Complétez pour les autres tables
    # N'oubliez pas les contraintes de clés étrangères !

    with engine.connect() as conn:
        conn.execute(text(create_customers))
        conn.execute(text(create_orders))
        conn.execute(text(create_sellers))
        conn.execute(text(create_products))
        conn.execute(text(create_order_items))
        
        # Exécutez les autres CREATE TABLE
        conn.commit()
create_tables()

## Traitements des données


In [ ]:
# Données vendeurs
df_sellers = df_list['olist_sellers_dataset']
df_sellers.isna().sum()
df_sellers.drop(columns=['seller_zip_code_prefix'], inplace=True)
print(df_sellers.head())

                          seller_id        seller_city seller_state
0  3442f8959a84dea7ee197c632cb2df15           campinas           SP
1  d1b65fc7debc3361ea86b5f14c68d2e2         mogi guacu           SP
2  ce3ad9de960102d0677a81f5d0bb7b2d     rio de janeiro           RJ
3  c0f3eea2e14555b6faeea3dd58c1b1c3          sao paulo           SP
4  51a04a8a6bdcb23deccc82b0b80742cf  braganca paulista           SP


In [92]:
# Données clients
df_customers = df_list['olist_customers_dataset']
df_customers.isna().sum()
df_customers.drop(columns=['customer_unique_id',
                           'customer_zip_code_prefix'],
                           inplace=True)
print(df_customers.head())


                        customer_id          customer_city customer_state
0  06b8999e2fba1a1fbc88172c00ba8bc7                 franca             SP
1  18955e83d337fd6b2def6b18a428ac77  sao bernardo do campo             SP
2  4e7b3e00288586ebd08712fdd0374a03              sao paulo             SP
3  b2b6027bc5c5109e529d4dc6358b12c3        mogi das cruzes             SP
4  4f2d8ab171c80ec8364f7c12e35b23ad               campinas             SP


In [94]:
# Données objets commandés

df_order_items = df_list['olist_order_items_dataset']
df_order_items.isna().sum()
df_order_items.drop(columns=['shipping_limit_date'], inplace=True)
print(df_order_items.head())

                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id   price  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   58.90   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36  239.90   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d  199.00   
3  7634da152a4610f1595efa32f14722fc  9d7a1d34a5052409006425275ba1c2b4   12.99   
4  ac6c3623068f30de03045865e4e10089  df560393f3a51e74553ab94004ba5c87  199.90   

   freight_value  
0          13.29  
1          19.93  
2          17.87  
3          12.79  
4          18.14  


In [ ]:
# Dataframe Commandes
df_orders = df_list['olist_orders_dataset']
df_orders.isna().sum()

# Suppression des colonnes non-utilisées
df_orders.drop(columns=['order_approved_at', 'order_estimated_delivery_date',
                        'order_delivered_carrier_date'], inplace=True)

# Renommage de la colonne 'order_purchase_timestamp' en 'order_date'
df_orders.rename(columns={'order_purchase_timestamp': 'order_date',
                           'order_delivered_customer_date': 'order_delivered_date'},
                           inplace=True)
df_orders[df_orders["order_delivered_date"].isnull()]\
         .groupby("order_status").size()

# Suppression des données manquantes dans la colonne 'order_delivered_date' 
df_orders = df_orders[df_orders['order_delivered_date'].notnull()]
print(df_orders)





In [ ]:


# Données produits Produits

df_products = df_list['olist_products_dataset']
df_products[df_products.isnull().any(axis=1)] # plusieurs données manquantes
df_products.rename(columns={"product_category_name": "product_category"}, inplace=True) # mise à jour du nom de colonne en vue d'un export sur la base de données crée
# Gestion des données manquantes dans la colonne 'product_category'
df_products['product_category'].fillna("uncategorized", inplace=True) 
df_products.dropna(subset=["product_weight_g"], inplace=True)
    # Suppression des colonnes inutiles
df_products.drop(columns=['product_name_lenght', 'product_description_lenght', 
                          'product_photos_qty', 'product_length_cm', 
                          'product_height_cm', 'product_width_cm'], inplace=True)


print(df_products)

/tmp/ipykernel_36171/1806966647.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_products['product_category'].fillna("uncategorized", inplace=True)


KeyError: "['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_length_cm', 'product_height_cm', 'product_width_cm'] not found in axis"

## Partie 3 : Requêtes SQL avancées


### 🔍 Analyses SQL à implémenter

#### 1. Analyse RFM (Récence, Fréquence, Montant)
```sql
-- Votre défi : Calculer les métriques RFM pour chaque client
WITH customer_metrics AS (
    SELECT
        c.customer_id,
        c.customer_state,
        -- Récence : jours depuis dernier achat
        -- Fréquence : nombre de commandes
        -- Montant : total dépensé
        
        -- Complétez cette requête CTE
        
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id, c.customer_state
)

-- Créez les segments RFM (Champions, Loyaux, À risque, etc.)
SELECT
    customer_id,
    customer_state,
    recency_score,
    frequency_score,
    monetary_score,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        -- Ajoutez les autres segments
        ELSE 'Others'
    END as customer_segment
FROM customer_metrics;
```

In [ ]:
#### 2. Analyse géographique des ventes

def geographic_sales_analysis():
    """
    Analysez les performances par état/région

    Requêtes à écrire :
    1. Top 10 des états par CA
    2. Croissance MoM par région
    3. Taux de conversion par ville
    4. Distance moyenne vendeur-acheteur
    """

    query_top_states = """
    -- Votre requête SQL ici
    -- Utilisez des JOINs et GROUP BY
    -- Calculez le CA, nombre de commandes, panier moyen
    """

    return pd.read_sql(query_top_states, engine)

#### 3. Analyse temporelle et saisonnalité
```sql
-- Détectez les patterns saisonniers
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
```

---

## Partie 4 : Analyse prédictive avec SQL

### 🔮 Modèles simples en SQL

In [ ]:
#### 1. Prédiction de churn

def churn_prediction_sql():
    """
    Identifiez les clients à risque de churn

    Indicateurs :
    - Pas d'achat depuis X jours
    - Baisse de fréquence d'achat
    - Diminution du panier moyen
    - Changement de comportement géographique
    """

    churn_query = """
    WITH customer_activity AS (
        -- Calculez les métriques d'activité récente
        -- Comparez avec l'historique du client
        -- Scorez le risque de churn
    )

    SELECT
        customer_id,
        days_since_last_order,
        order_frequency_trend,
        monetary_trend,
        churn_risk_score,
        CASE
            WHEN churn_risk_score > 0.7 THEN 'High Risk'
            WHEN churn_risk_score > 0.4 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as churn_segment
    FROM customer_activity;
    """

    return pd.read_sql(churn_query, engine)


#### 2. Recommandations produits
```sql
-- Market Basket Analysis simplifié
WITH product_pairs AS (
    SELECT
        oi1.product_id as product_a,
        oi2.product_id as product_b,
        COUNT(*) as co_purchase_count
    FROM order_items oi1
    JOIN order_items oi2 ON oi1.order_id = oi2.order_id
    WHERE oi1.product_id != oi2.product_id
    GROUP BY oi1.product_id, oi2.product_id
    HAVING COUNT(*) >= 10  -- Seuil minimum
)

SELECT
    product_a,
    product_b,
    co_purchase_count,
    co_purchase_count::float / total_a.count as confidence
FROM product_pairs pp
JOIN (
    SELECT product_id, COUNT(*) as count
    FROM order_items
    GROUP BY product_id
) total_a ON pp.product_a = total_a.product_id
ORDER BY confidence DESC;
```

---

## Partie 5 : Intégration avec les APIs météo

### 🌤️ Croisement données météo/ventes
```python
def weather_sales_correlation():
    """
    Correlez vos données météo du Notebook 1 avec les ventes
    
    Hypothèses à tester :
    1. Les ventes de certaines catégories augmentent-elles avec la pluie ?
    2. Y a-t-il un impact de la température sur les achats ?
    3. Les livraisons sont-elles impactées par la météo ?
    """
    
    # Récupérez les données météo historiques pour les villes brésiliennes
    weather_query = """
    SELECT DISTINCT customer_city, customer_state
    FROM customers
    WHERE customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC')
    ORDER BY customer_city;
    """
    
    cities = pd.read_sql(weather_query, engine)
    
    # Intégrez avec l'API météo
    # Analysez les corrélations
    
    pass
```

### 📊 Dashboard géo-temporel
```python
def create_geotemporal_dashboard():
    """
    Créez un dashboard interactif combinant :
    - Carte des ventes par région
    - Évolution temporelle avec météo
    - Segments clients géolocalisés
    - Prédictions par zone géographique
    """
    pass
```

---
## 🏆 Livrables finaux

### 📈 Rapport d'analyse complet
1. **Segmentation RFM (Recency, Frenquency, Monetary) ** : 5-7 segments avec caractéristiques
2. **Analyse géographique**  : Performances par région + recommandations
3. **Prédictions churn** : Liste des clients à risque + actions
4. **Recommandations produits** : Top 10 des associations
5. **Impact météo** : Corrélations significatives identifiées

### 🚀 Pipeline automatisé
```python
def automated_analysis_pipeline():
    """
    Pipeline qui :
    1. Se connecte à la DB
    2. Exécute toutes les analyses
    3. Met à jour les segments clients
    4. Génère le rapport automatiquement
    5. Envoie des alertes si nécessaire
    """
    pass
```

---

## 🎓 Auto-évaluation

- [ ] **Connexion DB** : PostgreSQL fonctionnelle
- [ ] **Requêtes complexes** : JOINs, CTEs, fonctions analytiques
- [ ] **Gestion des erreurs** : Connexions robustes
- [ ] **Performance** : Requêtes optimisées avec index
- [ ] **Intégration** : SQL + Python + APIs
- [ ] **Insights actionables** : Recommandations business claires

### 🔗 Préparation au Notebook 3
Le prochain notebook portera sur NoSQL (MongoDB) avec des données de réseaux sociaux et d'IoT, en temps réel.

### 💡 Bases de données alternatives
- **PlanetScale** : MySQL serverless gratuit
- **MongoDB Atlas** : 512MB gratuit
- **FaunaDB** : Base multi-modèle gratuite
- **Hasura Cloud** : GraphQL + PostgreSQL